# Demo: Phase transition

In [1]:
from fastsir import DiscreteSIR as sir
import numpy as np
import networkx as nx

## ER graph, simple contagion

In [2]:
#Network parameters
N = 1000
mean_degree = 10
p = mean_degree/(N-1)
G = nx.erdos_renyi_graph(N,p)
max_degree = np.max([G.degree(n) for n in G])
edgelist = list(G.edges())

In [3]:
#contagion parameters
recovery_probability = 1
infection_func = lambda q,k: 1-(1-q)**k
qlist = np.linspace(0.05,0.2,20)

In [4]:
#simulation parameters
period = 10 #simulate until no infected node
nb_sample = 1

### Simulation

In [5]:
R_arr = np.zeros((qlist.shape[0],nb_sample)) #results output for order parameter

In [6]:
#test
infection_probability = infection_func(qlist[2],np.arange(max_degree+1))
process = sir(edgelist,recovery_probability,infection_probability)
initial_infected_nodes = {np.random.randint(N)}
process.infect_node_set(initial_infected_nodes)

In [7]:
process.evolve(2)
process.get_current_macro_state()

(2.0, 997, 1, 2)

In [8]:
process.get_macro_state_vector()

[(0.0, 999, 1, 0), (1.0, 998, 1, 1), (2.0, 997, 1, 2)]

In [9]:
for i,q in enumerate(qlist):
    print(i)
    #define process object for each q
    infection_probability = infection_func(q,np.arange(max_degree+1))
    process = sir(edgelist,recovery_probability,infection_probability)
    #get sample
    for j in range(nb_sample):
        print(j)
        initial_infected_nodes = {np.random.randint(N)}
        process.infect_node_set(initial_infected_nodes)
        process.evolve(period)
        R_arr[i,j] = process.get_current_macro_state()[3] #get number of recovered
        process.reset()

0
0
1
0
2
0


RuntimeError: Recovery attempt: the node is not infected

In [ ]:
R_arr